# Imports

In [2]:
import time
import os
import pickle
import torch
import speechbrain as sb
from speechbrain.dataio.dataio import read_audio
from loquacious_set_prepare import load_datasets

# from tqdm import tqdm
from tqdm.notebook import tqdm
from hyperpyyaml import load_hyperpyyaml
from speechbrain.dataio.sampler import DynamicBatchSampler

import torchaudio
import torchaudio.transforms as T

import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# speaker embeddings
from pyannote.audio import Inference
from pyannote.audio import Model

# !pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer


# Get CSVs

In [13]:
pth_sm = "/users/rwhetten/LeBenchmark/sm/train.csv"
lb_sm = pd.read_csv(pth_sm)

In [14]:
lb_sm.duration.max(), lb_sm.duration.min()

(20.0, 10.0)

# Remove Douplicates IDs from Large

In [15]:
pth = "/users/rwhetten/LeBenchmark/xlg_20/train.csv"
lb = pd.read_csv(pth)
lb.head()

,ID,wav,start,stop,duration
0,2567/3319_2567_000001.wav0.0,/corpus/LeBenchmark/mls_french_flowbert/gpfswo...,0,242080,15.13
1,2567/3319_2567_000000.wav0.0,/corpus/LeBenchmark/mls_french_flowbert/gpfswo...,0,264000,16.50
2,1565/2284_1565_000000.wav0.0,/corpus/LeBenchmark/mls_french_flowbert/gpfswo...,0,190080,11.88
3,1565/2284_1565_000008.wav0.0,/corpus/LeBenchmark/mls_french_flowbert/gpfswo...,0,298880,18.68
4,1565/2284_1565_000013.wav0.0,/corpus/LeBenchmark/mls_french_flowbert/gpfswo...,0,204960,12.81


In [16]:
lb.duration.sum() / 3600

14563.676404843749

In [17]:
lb.duration.max(), lb.duration.min()

(20.0, 0.0009375)

In [19]:
ct=lb.groupby(['ID']).size().reset_index(name='count')
ct[ct['count'] > 1]

,ID,count
355,008/afc-gabon_16.06.11_008_read_0080.wav0.0,2
363,008/afc-gabon_16.06.11_008_read_0088.wav0.0,2
364,008/afc-gabon_16.06.11_008_read_0089.wav0.0,2
725,015/afc-gabon_16.06.11_015_read_0080.wav0.0,2
733,015/afc-gabon_16.06.11_015_read_0088.wav0.0,2
...,...,...
3850458,split/mirbeau_7.wav20.0,2
3850479,split/mirbeau_8.wav0.0,2
3850480,split/mirbeau_8.wav20.0,2
3850501,split/mirbeau_9.wav0.0,2


In [20]:
lb['ID_count'] = lb.groupby('ID').cumcount()

In [21]:
lb['Unique_ID'] = lb.apply(
    lambda row: str(row['ID']) if row['ID_count'] == 0 else f"{row['ID']}_{row['ID_count']+1}",
    axis=1
)

# Drop the helper column if you don't need it
lb = lb.drop(columns='ID_count')

In [25]:
lb.ID = lb.Unique_ID
lb = lb.drop(columns='Unique_ID')

In [26]:
ct=lb.groupby(['ID']).size().reset_index(name='count')
ct[ct['count'] > 1]

,ID,count


In [27]:
lb.duration.sum() / 3600

14563.676404843749

In [29]:
lb_filtered = lb[lb.duration > 2]

In [30]:
lb_filtered.duration.sum() / 3600

14503.947984357652

In [31]:
lb_filtered.to_csv(pth)

## Reread in to double check

In [32]:
pth = "/users/rwhetten/LeBenchmark/xlg_20/train.csv"
lb = pd.read_csv(pth)
lb.head()

,Unnamed: 0,ID,wav,start,stop,duration
0,0,2567/3319_2567_000001.wav0.0,/corpus/LeBenchmark/mls_french_flowbert/gpfswo...,0,242080,15.13
1,1,2567/3319_2567_000000.wav0.0,/corpus/LeBenchmark/mls_french_flowbert/gpfswo...,0,264000,16.50
2,2,1565/2284_1565_000000.wav0.0,/corpus/LeBenchmark/mls_french_flowbert/gpfswo...,0,190080,11.88
3,3,1565/2284_1565_000008.wav0.0,/corpus/LeBenchmark/mls_french_flowbert/gpfswo...,0,298880,18.68
4,4,1565/2284_1565_000013.wav0.0,/corpus/LeBenchmark/mls_french_flowbert/gpfswo...,0,204960,12.81


In [34]:
lb.duration.sum() / 3600

14503.947984357652